In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Get today's date in YYYY-MM-DD format
today = datetime.today().strftime("%Y-%m-%d")

# MLB Stats API endpoint for schedule/games
url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}"

# Send request to API
response = requests.get(url)
data = response.json()

# Extract games
games = data["dates"][0]["games"] if data["dates"] else []

# Format for DataFrame
game_list = []
for game in games:
    home = game["teams"]["home"]["team"]["name"]
    away = game["teams"]["away"]["team"]["name"]
    home_score = game["teams"]["home"].get("score", "-")
    away_score = game["teams"]["away"].get("score", "-")
    status = game["status"]["abstractGameState"]  # e.g. Preview, Live, Final
    inning = game["status"].get("inning", "-")

    game_list.append({
        "Away Team": away,
        "Home Team": home,
        "Away Score": away_score,
        "Home Score": home_score,
        "Status": status,
        "Inning": inning
    })

# Create DataFrame
df = pd.DataFrame(game_list)

# Display in Jupyter
df


,Away Team,Home Team,Away Score,Home Score,Status,Inning
0,Texas Rangers,Baltimore Orioles,-,-,Preview,-
1,Athletics,Detroit Tigers,-,-,Preview,-
2,Toronto Blue Jays,Cleveland Guardians,-,-,Preview,-
3,Atlanta Braves,New York Mets,-,-,Preview,-
4,New York Yankees,Cincinnati Reds,-,-,Preview,-
5,Tampa Bay Rays,Kansas City Royals,-,-,Preview,-
6,Seattle Mariners,Minnesota Twins,-,-,Preview,-
7,Arizona Diamondbacks,Chicago White Sox,-,-,Preview,-
8,Pittsburgh Pirates,Milwaukee Brewers,-,-,Preview,-
9,Chicago Cubs,St. Louis Cardinals,-,-,Preview,-
